# Notebook for developing code to simulate the SiPM pulses

In [1]:
import os
import numpy as np
import pandas as pd

np.random.seed(0)

In [2]:
# class Particle:
#     """ """
#     def __init__(self, type: str = 'MIP', pos: np.array = np.zeros(4), velocity: np.array = np.zeros(3)):
#         self.type     = type
#         self.pos      = pos
#         self.velocity = velocity
#         self.energy   = None
    
    
#     def __repr__(self):
#         return "<Particle type: %s, pos: %s, velocity: %s, energy: %s>" % (self.type, self.pos, self.velocity, self.energy)
    
#     def get_type(self):
#         return self.type
    
#     def get_pos(self):
#         return self.pos
# class MIP(Particle):
    
    
# class Photon(Particle):
    

In [3]:
type(np.zeros(8))

numpy.ndarray

In [183]:
class ParticleGun:
    
    def __init__(self, ptype: int = 0):
        self.ptype = ptype
        self.t_dist = lambda n = 1: np.random.uniform(0.1e-6, 0.9e-6) if n is 1 else np.random.uniform(0.1e-6, 0.9e-6, n)
        
    def generate(self, n: int = 1) -> np.ndarray:
        """ 
        Generates a new particle represented by a numpy.ndarray of length 10:
            [0]: particle_idx
            [1]: mother_idx
            [2]: Particle type, 0 = MIP, 10 = scintillation photon
            [3]: x
            [4]: y
            [5]: z
            [6]: t
            [7]: v_x
            [8]: v_y
            [9]: v_z
        """
        particles = np.zeros((n,10))
        particles[:,0] += np.arange(n)
        particles[:,6] += self.t_dist(n)

        return particles

In [247]:
class Scintillator():
    
    def __init__(self):

        self.t_dist = lambda : 0
        self.n_dist = lambda : 3  
            
    def generate(self, primaries: np.ndarray)-> np.ndarray:
        
        idx = max(primaries[:,0])
        
        photons = np.zeros((0,10))
        
        for primary in primaries:
            n_daughters = self.n_dist()
            for _ in range(n_daughters):
                idx += 1
                photon = np.zeros(10)
                photon[0] = idx
                photon[1] = primary[0]
                photon[6] = primary[6] + self.t_dist()
                photons = np.vstack([photons,photon])
        
        return photons
        

In [248]:
t_dist = lambda n = 1: np.random.uniform(0, 10e-9) if n is 1 else np.random.uniform(0, 10e-9, n)
t_dist(2)

array([1.52354706e-09, 4.17486375e-09])

In [249]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array( [6, 5, 4])
np.vstack([a,b])

array([[1, 2, 3],
       [4, 5, 6],
       [6, 5, 4]])

In [266]:
class SiPM():
    
    def __init__(self):
        
    def generate(self, photons: np.ndarray)-> np.ndarray:

IndentationError: expected an indented block (<ipython-input-266-92595a8faac9>, line 6)

In [251]:
class Electronics():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-251-888878041caa>, line 2)

In [252]:
class Digitizer():

SyntaxError: unexpected EOF while parsing (<ipython-input-252-55dd2924e582>, line 1)

In [263]:
class Event():

    def __init__(self, idx: int = -1):
        
        self.evt_idx = idx
        self.primaries = []
        self.scintillation_photons = []
        self.sipm_waveform = None
        self.electronics_waveform = None
        self.digitized_waveform = None
        
        columns = ['primaries_file',
                   'scintillation_photons_file',
                   'digitized_wf_file',
                   'n_particles',
                   'n_photons']
        
        data    = [str(self.evt_idx).zfill(7) + '_primaries.csv',
                   str(self.evt_idx).zfill(7) + '_scintillation_photons.csv',
                   str(self.evt_idx).zfill(7) + '_digitzed_wf.csv',
                   np.nan,
                   np.nan]
        
        self.meta    = pd.Series(data, index = columns) 
    
    def get_meta(self):
        return self.meta
    
    def set_primaries(self, primaries):
        self.primaries = primaries
    
    def get_primaries(self):
        return self.particles
    
    def set_photons(self, photons):
        self.scintillation_photons = photons
    
    def get_photons(self):
        return self.scintillation_photons

    def save_event(self, root_dir: str):
        """ Save primary particles and scintillation photons in separate files."""
        
        columns = ['particle_idx',
                   'mother_idx',
                   'type',
                   'x',
                   'y',
                   'z',
                   't',
                   'v_x',
                   'v_y',
                   'v_z']
        
        # Store the primary particles
        primaries_df = pd.DataFrame(columns = columns)
        
        for primary in self.primaries:
            se = pd.Series(primary, index = columns)
            primaries_df = primaries_df.append(se, ignore_index=True)

        primaries_df.to_csv(os.path.join(root_dir, self.meta['primaries_file']))
        
        # Store the scintillation photons
        photons_df = pd.DataFrame(columns = columns)
        
        for photon in self.scintillation_photons:
            se = pd.Series(photon, index = columns)
            photons_df = photons_df.append(se, ignore_index=True)

        photons_df.to_csv(os.path.join(root_dir, self.meta['scintillation_photons_file']))
        

In [264]:
class SimulationHandler:
    
    def __init__(self):
        
        self.idx          = -1
        self.primary_gun  = ParticleGun(ptype=0)
        self.scintillator = Scintillator()
        self.sipm         = SiPM()
        self.electronics  = None
        self.digitizer    = None
        
        self.events = []
        
        columns = ['primaries_file',
                   'scintillation_photons_file',
                   'digitized_wf_file',
                   'n_particles',
                   'n_photons']
        
        self.events_df = pd.DataFrame(columns = columns)
    
    def simulate_event(self):
        
        self.idx += 1
        event = Event(self.idx)
        primaries = self.primary_gun.generate(n=1)
        event.set_primaries(primaries)
        scintillation_photons = self.scintillator.generate(primaries)
        event.set_photons(scintillation_photons)
        sipm_wf = self.sipm.generate()
        
        self.events.append(event)
        

        
#     def simulate_and_save_event(self):
    """ TODO IMPLEMENT"""
    
    
    def save_events(self, root_dir):
        
        for event in self.events:
            
            meta = event.get_meta()
            
            # save particle_file
            # save photon_file
            # save waveform_file
            self.events_df = self.events_df.append(meta, ignore_index=True)
            event.save_event(root_dir)
            
        self.events_df.to_csv(os.path.join(root_dir,'test.csv'))
             

In [265]:
handler = SimulationHandler()

handler.simulate_event()
handler.simulate_event()

# handler.events[1].primaries
handler.save_events(r'C:\Users\Miroslav Gabriel\Workspace\claws-neural-net-discriminator\data\test')